In [75]:
import json
import requests
import urllib.request
import pandas
import tarfile

In [99]:
r = requests.get('https://api.github.com/repos/spack/spack/releases')
#pandas.json_normalize(r, max_level=0)

fields = 'name', 'published_at', 'tarball_url'
df = pandas.DataFrame(map(lambda _: map(lambda x:_[x], fields), r.json()), columns=fields)[:21]
df.published_at=pandas.to_datetime(df.published_at)
df['release_date']=df.published_at.dt.date

In [100]:
import shutil, os, subprocess

numpkg=[]
for index, row in df.iterrows():
    if index>=0:
        ftpstream = urllib.request.urlopen(row['tarball_url'])
        with tarfile.open(fileobj=ftpstream, mode="r|gz") as thetarfile:
            prefix=os.path.join(os. getcwd(),thetarfile.firstmember.name)
            thetarfile.extractall()
            spack_init_script=os.path.join(prefix, 'share','spack','setup-env.sh')
#             print(os.path.isfile(spack_init_script))
#             cmd='{} && spack list | wc -l'.format(spack_init_script)
            process = subprocess.Popen('ls {} | wc -l'.format(os.path.join(prefix, 'var', 'spack', 'repos', 'builtin', 'packages')),stdout=subprocess.PIPE, shell=True)
            numpkg.append(int(process.communicate()[0].strip()))
#             print(numpkg[-1])
            shutil.rmtree(prefix, ignore_errors=True)
df['numpkg']=numpkg
df

,name,published_at,tarball_url,release_date,numpkg
0,v0.16.1,2021-02-23 00:07:01+00:00,https://api.github.com/repos/spack/spack/tarba...,2021-02-23,5064
1,v0.16.0,2020-11-18 14:07:34+00:00,https://api.github.com/repos/spack/spack/tarba...,2020-11-18,5054
2,v0.15.4,2020-08-13 14:18:47+00:00,https://api.github.com/repos/spack/spack/tarba...,2020-08-13,4338
3,v0.15.3,2020-07-28 09:14:14+00:00,https://api.github.com/repos/spack/spack/tarba...,2020-07-28,4338
4,v0.15.2,2020-07-23 23:47:20+00:00,https://api.github.com/repos/spack/spack/tarba...,2020-07-23,4338
5,v0.15.1,2020-07-11 01:25:45+00:00,https://api.github.com/repos/spack/spack/tarba...,2020-07-11,4338
6,v0.15.0,2020-07-01 00:01:21+00:00,https://api.github.com/repos/spack/spack/tarba...,2020-07-01,4338
7,v0.14.2,2020-04-15 22:15:00+00:00,https://api.github.com/repos/spack/spack/tarba...,2020-04-15,3908
8,v0.14.1,2020-03-20 20:14:18+00:00,https://api.github.com/repos/spack/spack/tarba...,2020-03-20,3908
9,v0.14.0,2020-02-24 01:30:48+00:00,https://api.github.com/repos/spack/spack/tarba...,2020-02-24,3908


In [ ]:
import altair as alt

source = df
# alt.Chart(source).mark_line(interpolate='step-after').encode(
alt.Chart(source).mark_line(point=True).encode(
    x='release_date',
    y='numpkg'
).transform_filter(
    alt.datum.symbol == 'GOOG'
)